In [1]:
import numpy as np 
import pandas as pd
import datetime
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import re # regex
from scipy.misc import imread
from wordcloud import WordCloud, STOPWORDS # Wordcloud: pip install wordcloud

%matplotlib inline

In [4]:
postulantes_educacion = pd.read_csv('../data/1_postulantes_educacion_train.csv')
postulantes_genero_edad = pd.read_csv('../data/2_postulantes_genero_edad_train.csv')
vistas = pd.read_csv('../data/3_vistas_train.csv')
postulaciones = pd.read_csv('../data/4_postulaciones_train.csv')
avisos_online = pd.read_csv('../data/5_avisos_online_train.csv')
avisos_detalle = pd.read_csv('../data/6_avisos_detalle_train.csv')

# Juntamos de los postulantes la informacion sobre educacion, genero, y edad

### Obtenemos el maximo nivel de educacion para cada postulante

In [7]:
postulantes_educacion.head(2)

,idpostulante,nombre,estado
0,8BkL,Universitario,En Curso
1,1d2B,Universitario,En Curso


In [24]:
len(postulantes_educacion)

568612

In [23]:
print('Existen datos de educacion de', len(postulantes_educacion.drop_duplicates(['idpostulante'], keep='last', inplace=False)), 'postulantes')

Existen datos de educacion de 366395 postulantes


In [8]:
def cuantificar_estudios(row):
    if (row['nombre'] == 'Doctorado'):
        row['nombre'] = 7
    if (row['nombre'] == 'Master'):    
        row['nombre'] = 6
    if (row['nombre'] == 'Posgrado'):    
        row['nombre'] = 5
    if (row['nombre'] == 'Universitario'):    
        row['nombre'] = 4
    if (row['nombre'] == 'Terciario/Técnico'):    
        row['nombre'] = 3
    if (row['nombre'] == 'Secundario'):    
        row['nombre'] = 2
    if (row['nombre'] == 'Otro'):    
        row['nombre'] = 1    
        
    if (row['estado'] == 'Graduado'):
        row['estado'] = 3
    if (row['estado'] == 'En Curso'):    
        row['estado'] = 2
    if (row['estado'] == 'Abandonado'):    
        row['estado'] = 1
        
    return row    

def descuantificar_estudios(row):
    if (row['nombre'] == 7):
        row['nombre'] = 'Doctorado'
    if (row['nombre'] == 6):    
        row['nombre'] = 'Master'
    if (row['nombre'] == 5):    
        row['nombre'] = 'Posgrado'
    if (row['nombre'] == 4):    
        row['nombre'] = 'Universitario'
    if (row['nombre'] == 3):    
        row['nombre'] = 'Terciario/Técnico'
    if (row['nombre'] == 2):    
        row['nombre'] = 'Secundario'
    if (row['nombre'] == 1):    
        row['nombre'] = 'Otro'    
        
    if (row['estado'] == 3):
        row['estado'] = 'Graduado'
    if (row['estado'] == 2):    
        row['estado'] = 'En Curso'
    if (row['estado'] == 1):    
        row['estado'] = 'Abandonado'
        
    return row    

In [40]:
postulantes_educacion = postulantes_educacion.apply(lambda row: cuantificar_estudios(row), axis=1)

In [41]:
# Obtenemos max de la columna nombre
idx = postulantes_educacion.groupby(['idpostulante'])['nombre'].transform(max) == postulantes_educacion['nombre']

In [42]:
# Sobre los anteriores, obtenemos los max de la columna estado
idx_postulantes_educacion_max = postulantes_educacion[idx].groupby(['idpostulante'])['estado'].transform(max) == postulantes_educacion[idx]['estado']

In [43]:
postulantes_educacion_max = postulantes_educacion[idx][idx_postulantes_educacion_max]
len(postulantes_educacion_max)

366395

In [44]:
# Verificamos, por ejemplo para este postulante debe traer el maximo nombre y estado
postulantes_educacion[postulantes_educacion['idpostulante'] == '1QLLO'] 

,idpostulante,nombre,estado
5,1QLLO,4,2
290,1QLLO,4,3


In [45]:
# Efectivamente lo hace
postulantes_educacion_max[postulantes_educacion_max['idpostulante'] == '1QLLO'] 

,idpostulante,nombre,estado
290,1QLLO,4,3


In [46]:
postulantes_educacion_max = postulantes_educacion_max.apply(lambda row: descuantificar_estudios(row), axis=1)

In [49]:
postulantes_educacion_max.head(2)

,idpostulante,nombre,estado
1,1d2B,Universitario,En Curso
3,6M3jr,Universitario,En Curso


### Join con genero y edad

In [50]:
len(postulantes_genero_edad)

408146

In [51]:
len(postulantes_genero_edad.drop_duplicates(['idpostulante'], keep='last', inplace=False))

408146

In [52]:
postulantes = pd.merge(postulantes_educacion_max, postulantes_genero_edad, on='idpostulante', how='inner')

In [53]:
len(postulantes)

366395

In [54]:
postulantes.head(2)

,idpostulante,nombre,estado,fechanacimiento,sexo
0,1d2B,Universitario,En Curso,1976-02-28,MASC
1,6M3jr,Universitario,En Curso,1975-11-03,MASC
